# Extracting points of interest

Import the necessary packages:
osmnx and spatialzosm 

In [1]:
import sys
import geopandas as gpd
from spatialzosm import Osmpoi  

Create an object of the class Osmpoi and initialize the parameters of the object

In [2]:
frankfurt = Osmpoi({
    "city": "Frankfurt am Main",
    "country": "Germany",
}) 
frankfurt.file_export='Frankfurt_POIs'
frankfurt.save_raw=True

You can extract the points of interest using `fetch_osm_points()`. A raw dataset of the points of interest will be retrieved

In [ ]:
frankfurt_pois = frankfurt.fetch_osm_points() 

The raw dataset can be cleaned and automatically filtered using `filter_osm_points() `

In [ ]:
osmdata = frankfurt.filter_osm_points(frankfurt_pois) 

Now we have a dataset of the points of interest in Frankfurt

# Generating home locations

We read the geospatial dataset about Frankfurt cotaining the information of its zone units (ZUs), which in this case are districts, and the population counts of each ZU. The input dataset is **Frankfurt_districts_pop.gpkg**

In [36]:
gdf = gpd.read_file('input_data/Frankfurt_districts_pop.gpkg',crs='EPSG:4326')
gdf.head()

,STTLNR,STTLNAME,Frankfurt_pop_field_3,geometry
0,1,Altstadt,4063,"MULTIPOLYGON (((8.68787 50.11416, 8.68789 50.1..."
1,2,Innenstadt,6491,"MULTIPOLYGON (((8.68683 50.12013, 8.68891 50.1..."
2,4,Westend-Süd,18798,"MULTIPOLYGON (((8.66070 50.12310, 8.66181 50.1..."
3,5,Westend-Nord,10026,"MULTIPOLYGON (((8.67113 50.13696, 8.67140 50.1..."
4,6,Nordend-West,30307,"MULTIPOLYGON (((8.69722 50.13692, 8.69669 50.1..."


We rename the columns by giving an appropriate name

In [37]:
new_ind = 'AZ'
gdf.rename(columns={'Frankfurt_pop_field_3':'pop','STTLNR':new_ind}, inplace=True)
gdf.set_index('AZ',inplace = True)
gdf.head()

,STTLNAME,pop,geometry
AZ,,,
1,Altstadt,4063,"MULTIPOLYGON (((8.68787 50.11416, 8.68789 50.1..."
2,Innenstadt,6491,"MULTIPOLYGON (((8.68683 50.12013, 8.68891 50.1..."
4,Westend-Süd,18798,"MULTIPOLYGON (((8.66070 50.12310, 8.66181 50.1..."
5,Westend-Nord,10026,"MULTIPOLYGON (((8.67113 50.13696, 8.67140 50.1..."
6,Nordend-West,30307,"MULTIPOLYGON (((8.69722 50.13692, 8.69669 50.1..."


## Generating locations over the zone units

We generate the locations across the areas of the ZUs uniformly

In [ ]:
POIs_area_uniform= frankfurt.create_houses_areas(gdf, crs='EPSG:4326',method='uniform',size=gdf['pop'].values)

We can also generate the locations across the areas of the ZUs using a normal distribution around a center point. The center points are the centroids of the ZUs shapes

In [ ]:
POIs_area_normal= frankfurt.create_houses_areas(gdf, crs='EPSG:4326',method='normal',size=gdf['pop'].values)

## Generating home locations along the street network

To generate home locations along the road network, we need the geospatial information of the road network. 
We can extract the road network from OSM using:

In [ ]:
street_network = frankfurt.fetch_osm_streets()

The datafile is saved or stored as a variable

Next, we load the extracted road network from file 

In [ ]:
gdf_streets = gpd.read_file('Frankfurt_POIs_streets.csv',GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO", crs='EPSG:4326')
gdf_streets.plot()

Join both datasets `gdf_streets` and `gdf` to have a dataset of the road newtork with the associated zone unit each road belongs to

In [ ]:
roads_region=gpd.sjoin(gdf_streets, gdf,how="inner",predicate='intersects')
roads_region.rename(columns={"index_right": new_ind},inplace=True)
pop_size = gdf.loc[roads_region[new_ind].unique()]

We generate home locations along the road network using the road network `roads_region` and the population of each zone unit `pop_size[pop]`

In [ ]:
POI_houses = frankfurt.create_houses_streets(roads_region,pop_size=pop_size['pop'],crs='EPSG:4326')

## Generating home locations within building geometries

To generate home locations along the road network, we need the geospatial information of the building shapes.
We can extract the road network from OSM using:

In [ ]:
buildings=frankfurt.fetch_osm_buildings()

We read the obtained dataset

In [ ]:
gdf_building = gpd.read_file('Frankfurt_POIs_buildings.csv',GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO", crs='EPSG:4326')

We filter the dataset to obtain only buildings that can allocate homes

In [ ]:
gdf_building['building']=gdf_building['building'].astype(str)
gdf_building=gdf_building[(gdf_building['building']=='yes') | (gdf_building['building']=='hotel')| (gdf_building['building']=='apartment')| (gdf_building['building']=='apartments')| (gdf_building['building']=='bungalow')| (gdf_building['building']=='cabin')| (gdf_building['building']=='dormitory')| (gdf_building['building']=='flats')| (gdf_building['building']=='house')|  (gdf_building['building']=='residential')| (gdf_building['building']=='boathouse')| (gdf_building['building']=='allotment_house')| (gdf_building['building']=='terrace')| (gdf_building['building']=='transformer_house')| (gdf_building['building']=='summer_house')| (gdf_building['building']=='houseboat')| (gdf_building['building']=='terrace')]

gdf_building=gdf_building[['building','geometry']]

Join both datasets `gdf_building` and `gdf` to have a dataset of the road newtork with the associated zone unit each road belongs to

In [ ]:
buildings_region=gpd.sjoin(gdf_building, gdf,how="left",predicate='within')
buildings_region.rename(columns={"index_right": new_ind},inplace=True)
buildings_region=buildings_region.dropna(subset=new_ind)
pop_size = gdf.loc[buildings_region[new_ind].unique()]

Generate the locations within the buildings. The generation is based on the types of buildings in **building** column of `buildings_region`

In [ ]:
building_type_column = 'building'
POI_houses = frankfurt.create_houses_buildings(buildings_region,new_ind,building_type_column,pop_size=pop_size['pop'])